## Taller Final - Corte 3: De los Datos Crudos al Insight Gerencial
### Sector: Industria Automotriz
#### Grupo 2: Juanita Merchán y Annie Bonilla

In [208]:
# Instalación de librerías necesarias
%pip install pandas openpyxl

# Asegurar que el pip esté actualizado
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [209]:
# Importar la librería pandas
import pandas as pd
import numpy as np

# Cargar el archivo tabla1_emc_general.xlsx
df_general = pd.read_excel('tabla1_emc_general.xlsx')

# Cargar el archivo tabla2_emc_vehiculos.xlsx
df_vehiculos = pd.read_excel('tabla2_emc_vehiculos.xlsx')

# Cargar el archivo tabla3_demografia_censo2018.xlsx
df_demografia = pd.read_excel('tabla3_demografia_censo2018_Og.xlsx')

# Mostrar las primeras filas para verificar
print("✅ Archivos originales cargados.")
print("-" * 50)
print("Datos de tabla1_emc_general.xlsx:")
print(df_general.head())
print("\n")
print("\nDatos de tabla2_emc_vehiculos.xlsx:")
print(df_vehiculos.head())
print("\n")
print("\nDatos de tabla3_demografia_censo2018.xlsx:")
print(df_demografia.head())
print("\n")

✅ Archivos originales cargados.
--------------------------------------------------
Datos de tabla1_emc_general.xlsx:
  Unnamed: 0 Unnamed: 1                                          Antioquia  \
0        Año        Mes  Total comercio minorista y vehículos en el dep...   
1        NaN        NaN                                                NaN   
2        NaN        NaN                                                NaN   
3       2025      Enero                                         123.665347   
4        NaN    Febrero                                         120.511263   

                                          Unnamed: 3  \
0  Comercio, mantenimiento y reparación de vehícu...   
1                                                NaN   
2                 4511. Vehículos automotores nuevos   
3                                         108.626431   
4                                         126.738337   

                                          Unnamed: 4  \
0                    

#### Ingenieria de Datos con Python y Pandas

In [210]:
# --- Definición de Funciones de Utilidad ---

def clean_column_names(df):
    """Estandariza y limpia los nombres de columnas de un DataFrame."""
    df = df.copy()
    df.columns = (
        df.columns.astype(str).str.strip()
                    .str.lower()
                    .str.replace(" ", "_", regex=False)
                    .str.replace("\n", "_", regex=False)
                    .str.replace("(", "", regex=False)
                    .str.replace(")", "", regex=False)
    )
    return df

def convert_numeric_columns(df, cols):
    """Convierte una lista de columnas a tipo numérico, forzando errores a NaN."""
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def fill_missing_median(df):
    """Rellena los valores NaN de las columnas numéricas con la mediana de la columna."""
    df = df.copy()
    num_cols = df.select_dtypes(include=["number"]).columns
    for c in num_cols:
        df[c] = df[c].fillna(df[c].median())
    return df

def capitalizar_meses(df):
    """Estandariza la capitalización de los meses para crear la columna Fecha."""
    df = df.copy()
    meses = {
        'enero': 'Enero', 'febrero': 'Febrero', 'marzo': 'Marzo',
        'abril': 'Abril', 'mayo': 'Mayo', 'junio': 'Junio',
        'julio': 'Julio', 'agosto': 'Agosto', 'septiembre': 'Septiembre',
        'octubre': 'Octubre', 'noviembre': 'Noviembre', 'diciembre': 'Diciembre'
    }
    df['mes'] = df['mes'].astype(str).str.strip().str.lower().map(meses)
    return df

##### Base de datos 1: Informativos Comercio al por Menor - Por departamentos (EMC)

In [211]:
file_path1 = 'tabla1_emc_general.xlsx'

# Se lee el archivo Excel y se carga en un DataFrame de Pandas.
# El bloque 'try-except' es una buena práctica para manejar errores si el archivo no existe.
try:
    df1 = pd.read_excel(file_path1)
    print("✅ Archivo 'tabla1_emc_general.xlsx' cargado exitosamente.")
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo '{file_path1}'. Asegúrate de que esté en la misma carpeta.")

# Mostramos las primeras 5 filas para tener una idea inicial de los datos.
print("\n--- Vista Previa de los Datos Crudos ('Sucios') ---")
df1.head()

✅ Archivo 'tabla1_emc_general.xlsx' cargado exitosamente.

--- Vista Previa de los Datos Crudos ('Sucios') ---


,Unnamed: 0,Unnamed: 1,Antioquia,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Otros departamentos,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63
0,Año,Mes,Total comercio minorista y vehículos en el dep...,"Comercio, mantenimiento y reparación de vehícu...",NaN,Comercio al por menor,NaN,NaN,NaN,NaN,...,NaN,NaN,Total comercio minorista y vehículos en el dep...,"Comercio, mantenimiento y reparación de vehícu...",NaN,Comercio al por menor,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Comercio al por menor en establecimientos espe...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comercio al por menor en establecimientos espe...,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,4511. Vehículos automotores nuevos,"4530- 4541. Partes, piezas (autopartes) y acce...",4711 - 4719 - 4721 -4722 - 4723 - 4724 - 4729 ...,4741 - 4742 - 4751 - 4752 - 4753 - 4754 - 4755...,4761 - 4762 - . 4769 - 4774. Artículos cultura...,4771 - 4772. Prendas de vestir y sus accesorio...,"4773. Productos farmacéuticos, medicinales, od...",...,"4773. Productos farmacéuticos, medicinales, od...",NaN,NaN,4511. Vehículos automotores nuevos,"4530- 4541. Partes, piezas (autopartes) y acce...",4711 - 4719 - 4721 -4722 - 4723 - 4724 - 4729 ...,4741 - 4742 - 4751 - 4752 - 4753 - 4754 - 4755...,4761 - 4762 - . 4769 - 4774. Artículos cultura...,4771 - 4772. Prendas de vestir y sus accesorio...,"4773. Productos farmacéuticos, medicinales, od..."
3,2025,Enero,123.665347,108.626431,134.459701,123.292346,139.810596,118.698212,108.320114,144.791796,...,147.478574,NaN,123.049701,110.331384,138.061059,121.41541,143.688045,144.49922,100.651032,130.540544
4,NaN,Febrero,120.511263,126.738337,130.970545,118.826733,127.039894,94.837225,87.849443,131.151392,...,140.099254,NaN,113.991154,113.478661,135.584995,112.35687,129.718159,108.474957,81.438478,107.829568


In [212]:
# Antes de limpiar, un buen analista siempre diagnostica el estado de los datos.

print("\n--- Diagnóstico Inicial del DataFrame ---")

# .info() nos da un resumen técnico: número de filas, columnas, tipos de datos y valores no nulos.
# Es la "radiografía" de nuestros datos.
print("1. Información General y Tipos de Datos:\n")

df1.info()

# .isnull().sum() nos dice exactamente cuántos valores nulos hay en cada columna.
print("\n2. Conteo de Valores Nulos por Columna:")
print(df1.isnull().sum())

# Creamos una copia del DataFrame para trabajar sobre ella, conservando el original.
# Esta es una práctica fundamental para no perder los datos crudos.
df1_clean = df1.copy()
df1_clean = df1_clean.round(3)


--- Diagnóstico Inicial del DataFrame ---
1. Información General y Tipos de Datos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 64 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           2 non-null      object 
 1   Unnamed: 1           10 non-null     object 
 2   Antioquia            10 non-null     object 
 3   Unnamed: 3           11 non-null     object 
 4   Unnamed: 4           10 non-null     object 
 5   Unnamed: 5           12 non-null     object 
 6   Unnamed: 6           10 non-null     object 
 7   Unnamed: 7           10 non-null     object 
 8   Unnamed: 8           10 non-null     object 
 9   Unnamed: 9           10 non-null     object 
 10  Unnamed: 10          0 non-null      float64
 11  Atlántico            10 non-null     object 
 12  Unnamed: 12          11 non-null     object 
 13  Unnamed: 13          10 non-null     object 
 14  Unnamed

In [213]:
def clean_df1_modified(file_path1):
    """
    CORRECCIÓN EMC GENERAL: Asegura la extracción de la serie '4511. Vehículos automotores nuevos'
    usando el MultiIndex.
    """
    df1_clean = pd.read_excel(file_path1, header=[0, 1, 2])
    
    cols_to_keep = [
        ('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Año'),
        ('Unnamed: 1_level_0', 'Unnamed: 1_level_1', 'Mes'),
    ]
    
    departamentos = ['Antioquia', 'Atlántico', 'Bogotá', 'Cundinamarca', 'Santander', 'Valle del Cauca', 'Otros departamentos']
    data_level = '4511. Vehículos automotores nuevos'
    
    for dept in departamentos:
        cols_to_keep.append((dept, 'Comercio, mantenimiento y reparación de vehículos automotores y motocicletas, sus partes, piezas y accesorios', data_level))
        
    df1_result = df1_clean[cols_to_keep].copy()
    
    new_cols = ['Año', 'Mes'] + departamentos
    df1_result.columns = new_cols
    
    df1_result['Año'] = df1_result['Año'].ffill()
    df1_result = df1_result.dropna(subset=['Mes']).reset_index(drop=True)
    
    numeric_cols = [c for c in new_cols if c not in ['Año', 'Mes']]
    df1_result = convert_numeric_columns(df1_result, numeric_cols)
    df1_result = fill_missing_median(df1_result)
    
    df1_result = capitalizar_meses(df1_result.rename(columns={'Mes': 'mes'}))
    df1_result['Fecha'] = df1_result['mes'] + '-' + df1_result['Año'].astype(int).astype(str)
    
    final_cols = ['Fecha'] + numeric_cols
    df1_final = df1_result[final_cols].round(16)
    
    return df1_final

##### Base de datos 2: Informativos Comercio de Vehículos - Por departamentos (EMC)

In [214]:
file_path2 = 'tabla2_emc_vehiculos.xlsx'

# Se lee el archivo Excel y se carga en un DataFrame de Pandas.
# El bloque 'try-except' es una buena práctica para manejar errores si el archivo no existe.
try:
    df2 = pd.read_excel(file_path2)
    print("✅ Archivo 'tabla2_emc_vehiculos.xlsx' cargado exitosamente.")
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo '{file_path2}'. Asegúrate de que esté en la misma carpeta.")

# Mostramos las primeras 5 filas para tener una idea inicial de los datos.
print("\n--- Vista Previa de los Datos Crudos ('Sucios') ---")
df2.head()

✅ Archivo 'tabla2_emc_vehiculos.xlsx' cargado exitosamente.

--- Vista Previa de los Datos Crudos ('Sucios') ---


,Tipo de vehículo,Unnamed: 1,Vehículos de origen nacional + importado,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,Automóviles particulares,NaN,Camperos y camionetas,NaN,Vehículos de transp. público,NaN,Vehículos de carga,NaN,Motocicletasa,NaN
1,NaN,NaN,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas,Unidades vendidas,Valor ventas
2,2025.0,Enero,3405,241201385,9471,1181596926,553,82856022,1068,247300066,76268,531886954
3,NaN,Febrero,4531,307182298,8910,1071946799,487,64792361,1296,233869642,83739,596627061
4,NaN,Marzo,4712,321924982,10712,1262303106,492,60273624,1281,198738588,85762,604323891


In [215]:
# Antes de limpiar, un buen analista siempre diagnostica el estado de los datos.

print("\n--- Diagnóstico Inicial del DataFrame ---")

# .info() nos da un resumen técnico: número de filas, columnas, tipos de datos y valores no nulos.
# Es la "radiografía" de nuestros datos.
print("1. Información General y Tipos de Datos:\n")

df2.info()

# .isnull().sum() nos dice exactamente cuántos valores nulos hay en cada columna.
print("\n2. Conteo de Valores Nulos por Columna:")
print(df2.isnull().sum())

# Creamos una copia del DataFrame para trabajar sobre ella, conservando el original.
# Esta es una práctica fundamental para no perder los datos crudos.
df2_clean = df2.copy()
df2_clean = df2_clean.round(3)


# Retirar mayusculas para facilitar el acceso al proceso de finaniciación
def capitalizar_meses_df2_clean(df2_clean):
    meses = {
        'enero': 'Enero','febrero': 'Febrero','marzo': 'Marzo',
        'abril': 'Abril','mayo': 'Mayo','junio': 'Junio',
        'julio': 'Julio','agosto': 'Agosto','septiembre': 'Septiembre',
        'setiembre': 'Septiembre','octubre': 'Octubre',
        'noviembre': 'Noviembre','diciembre': 'Diciembre'
    }


--- Diagnóstico Inicial del DataFrame ---
1. Información General y Tipos de Datos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Tipo de vehículo                          1 non-null      float64
 1   Unnamed: 1                                9 non-null      object 
 2   Vehículos de origen nacional + importado  11 non-null     object 
 3   Unnamed: 3                                10 non-null     object 
 4   Unnamed: 4                                11 non-null     object 
 5   Unnamed: 5                                10 non-null     object 
 6   Unnamed: 6                                11 non-null     object 
 7   Unnamed: 7                                10 non-null     object 
 8   Unnamed: 8                                11 non-null     object 
 9   Unnamed: 9                

In [216]:
def clean_df2_modified(file_path2):
    """
    CORRECCIÓN EMC VEHÍCULOS: Construye los nombres de columna exactos, respetando
    los espacios al final en algunos nombres de columna de salida solicitados.
    """
    # Cargar con los encabezados que contienen el tipo de vehículo (nivel 1) y la métrica (nivel 2)
    df_raw = pd.read_excel(file_path2, header=[1, 2])
    
    # Mapeo de nombres para la salida
    final_col_names_map = {
        'Unidades vendidas': 'Q vendidas',
        'Valor ventas': 'Valor ventas'
    }
    
    # Generar los nuevos nombres de columna con el formato exacto requerido
    new_cols = ['Fecha']
    
    # Iterar sobre las columnas y construir el nuevo nombre
    for col in df_raw.columns:
        # Saltar las columnas de Año/Mes (que son Unnamed en el nivel 1)
        if isinstance(col[0], str) and 'Unnamed' not in col[0]:
            vehiculo = col[0].strip().replace('\n', ' ')
            medida = col[1].strip().replace('\n', ' ')
            
            # Estandarización del nombre del vehículo y el sufijo para coincidir exactamente con el output deseado
            if 'Automóviles particulares' in vehiculo:
                vehiculo = 'Automóviles particulares'
            elif 'Camperos y camionetas' in vehiculo:
                vehiculo = 'Camperos y camionetas'
            elif 'Vehículos de transp. público' in vehiculo:
                vehiculo = 'Vehículos de transp. público ' # REQUIERE ESPACIO AL FINAL
            elif 'Vehículos de carga' in vehiculo:
                vehiculo = 'Vehículos de carga'
            elif 'Motocicletasa' in vehiculo:
                vehiculo = 'Motocicletasa ' # REQUIERE ESPACIO AL FINAL
                
            new_col_name = f"{final_col_names_map.get(medida, medida)} - {vehiculo}"
            new_cols.append(new_col_name)

    # Limpiar el DataFrame de filas de encabezado (asumiendo que los datos útiles inician en la fila 4)
    df2_result = df_raw.iloc[1:].copy()
    
    # Asignar nuevos nombres a las columnas de datos
    df2_result.columns = ['Tipo de vehículo', 'Unnamed: 1'] + new_cols[1:]
    
    # Procesar 'Tipo de vehículo' (Año) y 'Unnamed: 1' (Mes)
    df2_result = df2_result.rename(columns={'Unnamed: 1': 'mes'})
    df2_result['Tipo de vehículo'] = df2_result['Tipo de vehículo'].ffill().astype(int).astype(str)
    df2_result = df2_result.dropna(subset=['mes']).reset_index(drop=True)

    df2_result = capitalizar_meses(df2_result)
    df2_result['Fecha'] = df2_result['mes'] + '-' + df2_result['Tipo de vehículo']
    
    # 2. Aplicar conversiones numéricas y seleccionar el orden final
    numeric_cols = new_cols[1:]
    df2_final = df2_result[['Fecha'] + numeric_cols].copy()
    df2_final = convert_numeric_columns(df2_final, numeric_cols)
    df2_final = fill_missing_median(df2_final)
    
    return df2_final

##### Base de datos 3: PERSONAS (Demográfico)

In [217]:
file_path3 = 'tabla3_demografia_censo2018_Og.xlsx'

# Se lee el archivo Excel y se carga en un DataFrame de Pandas.
# El bloque 'try-except' es una buena práctica para manejar errores si el archivo no existe.
try:
    df3 = pd.read_excel(file_path3)
    print("✅ Archivo 'tabla3_demografia_censo2018.xlsx' cargado exitosamente.")
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo '{file_path3}'. Asegúrate de que esté en la misma carpeta.")

# Mostramos las primeras 5 filas para tener una idea inicial de los datos.
print("\n--- Vista Previa de los Datos Crudos ('Sucios') ---")
df3.head()

✅ Archivo 'tabla3_demografia_censo2018.xlsx' cargado exitosamente.

--- Vista Previa de los Datos Crudos ('Sucios') ---


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,2018,NaN,NaN,NaN,NaN
1,"Grupos de edad, departamento y Áreas (Total, ...",NaN,NaN,Sexo,NaN
2,NaN,NaN,Total,Hombre,Mujer
3,Total Nacional,Total,44164417,21570493,22593924
4,NaN,NaN,3037781,1555605,1482176


In [218]:
# Antes de limpiar, un buen analista siempre diagnostica el estado de los datos.

print("\n--- Diagnóstico Inicial del DataFrame ---")

# .info() nos da un resumen técnico: número de filas, columnas, tipos de datos y valores no nulos.
# Es la "radiografía" de nuestros datos.
print("1. Información General y Tipos de Datos:\n")

df3.info()

# .isnull().sum() nos dice exactamente cuántos valores nulos hay en cada columna.
print("\n2. Conteo de Valores Nulos por Columna:")
print(df3.isnull().sum())

# Creamos una copia del DataFrame para trabajar sobre ella, conservando el original.
# Esta es una práctica fundamental para no perder los datos crudos.
df3_clean = df3.copy()
df3_clean = df3_clean.round(3)


--- Diagnóstico Inicial del DataFrame ---
1. Información General y Tipos de Datos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2587 entries, 0 to 2586
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  36 non-null     object
 1   Unnamed: 1  136 non-null    object
 2   Unnamed: 2  2585 non-null   object
 3   Unnamed: 3  2586 non-null   object
 4   Unnamed: 4  2585 non-null   object
dtypes: object(5)
memory usage: 101.2+ KB

2. Conteo de Valores Nulos por Columna:
Unnamed: 0    2551
Unnamed: 1    2451
Unnamed: 2       2
Unnamed: 3       1
Unnamed: 4       2
dtype: int64


In [219]:
def clean_df3_modified(file_path3):
    """
    CORRECCIÓN DEMOGRAFÍA: Filtra solo la línea de 'Total' general para los 6 departamentos
    y elimina cualquier desagregación geográfica.
    """
    
    # 1. Cargar el archivo, asumiendo el encabezado en la fila 3 (header=2)
    df3_clean = pd.read_excel(file_path3, header=2) 
    
    # 2. Renombrar las columnas de interés
    cols_base = ['Departamento', 'Tipo_Area', 'Total', 'Hombre', 'Mujer']
    
    # Si hay más columnas (ej. desagregación por edad que no se eliminó completamente)
    df3_clean.columns = cols_base + [f'Col_{i}' for i in range(len(df3_clean.columns) - len(cols_base))]
    
    # Seleccionar solo las columnas de interés
    df3_final = df3_clean[cols_base].copy()
    
    # 3. Limpieza y estandarización
    df3_final['Departamento'] = df3_final['Departamento'].astype(str).str.strip().ffill()
    df3_final['Tipo_Area'] = df3_final['Tipo_Area'].astype(str).str.strip().str.lower()
    
    # Eliminar filas con NaN en los totales (ej. filas de metadatos)
    df3_final = df3_final.dropna(subset=['Total', 'Hombre', 'Mujer']).copy()
    
    # 4. FILTRO CRÍTICO: ELIMINAR DESAGREGACIONES GEOGRÁFICAS
    
    # 4.1. Mantener solo las filas que dicen 'total' en la columna 'Tipo_Area'
    df3_final = df3_final[
        df3_final['Tipo_Area'].str.contains('total', na=False)
    ].copy()
    
    # 4.2. Eliminar el "Total Nacional" si persiste
    df3_final = df3_final[
        ~df3_final['Departamento'].str.lower().str.contains('total nacional', na=False)
    ].reset_index(drop=True)
    
    # 5. Aplicar formato y filtro final
    
    # Eliminar la columna Tipo_Area y renombrar
    df3_final = df3_final.drop(columns=['Tipo_Area']).copy()
    
    # Estandarizar el nombre del departamento
    df3_final['Departamento'] = df3_final['Departamento'].str.title()
    df3_final['Departamento'] = df3_final['Departamento'].str.replace(' Del ', ' del ', regex=False)
    
    # 6. Filtrar por los seis departamentos solicitados
    target_depts = ['Antioquia', 'Atlántico', 'Bogotá', 'Cundinamarca', 'Santander', 'Valle Del Cauca']
    df3_final = df3_final[df3_final['Departamento'].isin(target_depts)].reset_index(drop=True)
    
    # 7. Limpieza y Formato final
    numeric_cols = ['Total', 'Hombre', 'Mujer']
    df3_final = convert_numeric_columns(df3_final, numeric_cols)
    df3_final = fill_missing_median(df3_final)
    df3_final[numeric_cols] = df3_final[numeric_cols].astype(int)

    return df3_final

In [221]:
import os

# --- Exportación de las Tablas Limpias ---

# Nota: Asumimos que df1_final, df2_final y df3_final ya contienen
# los DataFrames limpios generados en el paso anterior.

print("\n--- Preparación para Exportación ---")

# Las variables dfX_final ya existen, por lo que la copia es segura.
df1_final = df1_final.copy() 
df2_final = df2_final.copy()
df3_final = df3_final.copy()

# Exportar a Excel
output_filename = 'EMC_General_LimpioF.xlsx'
df1_final.to_excel(output_filename, index=False)

output_filename2 = 'EMC_Vehiculos_LimpioF.xlsx'
df2_final.to_excel(output_filename2, index=False)

output_filename3 = 'Demografia_Censo2018_LimpioF.xlsx'
df3_final.to_excel(output_filename3, index=False)

print("✅ Archivos exportados exitosamente en formato Excel (.xlsx).")


--- Preparación para Exportación ---
✅ Archivos exportados exitosamente en formato Excel (.xlsx).


In [ ]:
# --- Visualización de los Resultados (Simulando la salida final) ---

print("### Resultado Final de la Base de Datos 1 (tabla1_emc_general - Copy.xlsx)")
print(df1_final)
print("-" * 50)

print("### Resultado Final de la Base de Datos 2 (tabla2_emc_vehiculos - Copy.xlsx)")
print(df2_final)
print("-" * 50)

print("### Resultado Final de la Base de Datos 3 (tabla3_demografia_censo2018 - Copy.xlsx)")
print(df3_final)

### Resultado Final de la Base de Datos 1 (tabla1_emc_general - Copy.xlsx)
   Unnamed: 0    Unnamed: 1  \
0         Año           Mes   
1         NaN           NaN   
2         NaN           NaN   
3        2025         Enero   
4         NaN       Febrero   
5         NaN         Marzo   
6         NaN         Abril   
7         NaN          Mayo   
8         NaN         Junio   
9         NaN         Julio   
10        NaN        Agosto   
11        NaN    Septiembre   

                                            Antioquia  \
0   Total comercio minorista y vehículos en el dep...   
1                                                 NaN   
2                                                 NaN   
3                                          123.665347   
4                                          120.511263   
5                                          128.330292   
6                                          123.002019   
7                                          135.420266   
8       